In [0]:
dbutils.fs.ls("/mnt/bronze/SalesLT/")

In [0]:
#df=spark.read.format("parquet").load("/mnt/bronze/SalesLT/Address")

In [0]:
#display(df.limit(10))

In [0]:
#from pyspark.sql.functions import date_format, from_utc_timestamp
#from pyspark.sql.types import TimestampType

#df1=df.withColumn("Modified Date",date_format(from_utc_timestamp(df['modifiedDate'].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
#display(df1.limit(10))

In [0]:
table_name = []
for i in dbutils.fs.ls('mnt/bronze/SalesLT/'):
    table_name.append(i.name.split('/')[0])
table_name

In [0]:
from pyspark.sql.functions import date_format, from_utc_timestamp
from pyspark.sql.types import TimestampType

for i in table_name:
    path = f"/mnt/bronze/SalesLT/{i}/{i}.parquet"
    df = spark.read.format("parquet").load(path)
    columns = df.columns

    # Format any date columns
    for col in columns:
        if "Date" in col or "date" in col:
            df = df.withColumn(
                col,
                date_format(
                    from_utc_timestamp(df[col].cast(TimestampType()), "UTC"),
                    "yyyy-MM-dd"
                )
            )

    # Write each table separately
    output = f"/mnt/silver/SalesLT/{i}/"
    df.write.format("delta").mode("overwrite").save(output)